## CNN

In [ ]:
from CNN.lenet import LeNet5
import torch
from torchvision import datasets, transforms
import quartz
from quartz import layers
import numpy as np

In [ ]:
model = LeNet5(10)
model.load_state_dict(torch.load("CNN/quartz-lenet.pth", map_location=torch.device('cpu')))
model.eval()

In [ ]:
def get_weights_biases(model):
    parameters = list(model.parameters())
    weights = [weight.detach().numpy() for weight in parameters[::2]]
    biases = [bias.detach().numpy() for bias in parameters[1::2]]
    return weights, biases

weights, biases = get_weights_biases(model)

In [ ]:
max_weight = np.max([weight.max() for weight in weights])
min_weight = np.min([weight.min() for weight in weights])

In [ ]:
print(max_weight)
print(min_weight)

In [ ]:
ratio = 255/128
old_min = min_weight
new_min = -ratio
old_range = (max_weight - old_min)  
new_range = (ratio - new_min)

In [ ]:
weights = [(((weight - old_min) * new_range) / old_range) + new_min for weight in weights]

## build model

In [ ]:
t_max = 2**9
input_dims = (1,32,32)
pool_kernel_size = [2,2]

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims),
    layers.ConvPool2D(weights=weights[0], biases=biases[0], pool_kernel_size=pool_kernel_size),
    layers.ConvPool2D(weights=weights[1], biases=biases[1], pool_kernel_size=pool_kernel_size),
    layers.Conv2D(weights=weights[2], biases=biases[2]),
    layers.Dense(weights=weights[3], biases=biases[3]),
    layers.Dense(weights=weights[4], biases=biases[4]),
    layers.MonitorLayer(),
])

In [ ]:
#loihi_model

In [ ]:
#loihi_model.check_block_delays(t_max, 2**3)
#loihi_model.print_core_layout(redo=True)

## load data

In [ ]:
transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor(),])
test_set = datasets.MNIST('./CNN/data', train=False, download=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=True)

In [ ]:
sample, target = next(iter(test_loader))
inputs = quartz.utils.decode_values_into_spike_input(sample.squeeze().detach().numpy().flatten(), t_max)
target

In [ ]:
output = loihi_model(inputs, t_max)

In [ ]:
print("target was {} and result was {}".format(target.numpy()[0], np.argmax(output)))

In [ ]:
output

In [ ]:
with open("results.txt", "a") as myfile:
    myfile.write("{},{}\n".format(target.numpy()[0],np.argmax(output)))